# Basic Text Classification with Naive Bayes
***
In the mini-project, you'll learn the basics of text analysis using a subset of movie reviews from the rotten tomatoes database. You'll also use a fundamental technique in Bayesian inference, called Naive Bayes. This mini-project is based on [Lab 10 of Harvard's CS109](https://github.com/cs109/2015lab10) class.  Please free to go to the original lab for additional exercises and solutions.

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from six.moves import range

# Setup Pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("poster")

# Table of Contents

* [Rotten Tomatoes Dataset](#Rotten-Tomatoes-Dataset)
    * [Explore](#Explore)
* [The Vector Space Model and a Search Engine](#The-Vector-Space-Model-and-a-Search-Engine)
    * [In Code](#In-Code)
* [Naive Bayes](#Naive-Bayes)
    * [Multinomial Naive Bayes and Other Likelihood Functions](#Multinomial-Naive-Bayes-and-Other-Likelihood-Functions)
    * [Picking Hyperparameters for Naive Bayes and Text Maintenance](#Picking-Hyperparameters-for-Naive-Bayes-and-Text-Maintenance)
* [Interpretation](#Interpretation)


## Rotten Tomatoes Dataset

In [ ]:
critics = pd.read_csv('./critics.csv')
#let's drop rows with missing quotes
critics = critics[~critics.quote.isnull()]
critics.head()

### Explore

In [ ]:
n_reviews = len(critics)
n_movies = critics.rtid.unique().size
n_critics = critics.critic.unique().size


print("Number of reviews: {:d}".format(n_reviews))
print("Number of critics: {:d}".format(n_critics))
print("Number of movies:  {:d}".format(n_movies))

In [ ]:
df = critics.copy()
df['fresh'] = df.fresh == 'fresh'
grp = df.groupby('critic')
counts = grp.critic.count()  # number of reviews by each critic
means = grp.fresh.mean()     # average freshness for each critic

means[counts > 100].hist(bins=10, edgecolor='w', lw=1)
plt.xlabel("Average Rating per critic")
plt.ylabel("Number of Critics")
plt.yticks([0, 2, 4, 6, 8, 10]);

<div class="span5 alert alert-info">
<h3>Exercise Set I</h3>
<br/>
<b>Exercise:</b> Look at the histogram above. Tell a story about the average ratings per critic. What shape does the distribution look like? What is interesting about the distribution? What might explain these interesting things?
</div>

Observation:
    
* The distribution looks to be a normal distribution with a negative skew - in other words the distribution is weighted to results > 0.5. 

* This might be because there's a binomial distribution "hidden" in here, as results that are > 0.5 are additionally classifed as "fresh" and those < 0.5 are classified as "rotten". 
* From the data it would seem that critics are more inclined to give films a "fresh" rating than a "rotten" one, even if that means giving a score on the 0.6 borderline. But then, if the critic decides that a film is indeed "rotten", they don't go in much for borderline scores (of 0.4) but prefer lower scores.
* It seems that critics generally try to be nice, and would rather be positive about a film, but that if they decide they can't be positive, then they're better off being really negative than mildly negative. If you're going to be brutal, might as not mince words

## The Vector Space Model and a Search Engine

All the diagrams here are snipped from [*Introduction to Information Retrieval* by Manning et. al.]( http://nlp.stanford.edu/IR-book/) which is a great resource on text processing. For additional information on text mining and natural language processing, see [*Foundations of Statistical Natural Language Processing* by Manning and Schutze](http://nlp.stanford.edu/fsnlp/).

Also check out Python packages [`nltk`](http://www.nltk.org/), [`spaCy`](https://spacy.io/), [`pattern`](http://www.clips.ua.ac.be/pattern), and their associated resources. Also see [`word2vec`](https://en.wikipedia.org/wiki/Word2vec).

Let us define the vector derived from document $d$ by $\bar V(d)$. What does this mean? Each document is treated as a vector containing information about the words contained in it. Each vector has the same length and each entry "slot" in the vector contains some kind of data about the words that appear in the document such as presence/absence (1/0), count (an integer) or some other statistic. Each vector has the same length because each document shared the same vocabulary across the full collection of documents -- this collection is called a *corpus*.

To define the vocabulary, we take a union of all words we have seen in all documents. We then just associate an array index with them. So "hello" may be at index 5 and "world" at index 99.

Suppose we have the following corpus:

`A Fox one day spied a beautiful bunch of ripe grapes hanging from a vine trained along the branches of a tree. The grapes seemed ready to burst with juice, and the Fox's mouth watered as he gazed longingly at them.`

Suppose we treat each sentence as a document $d$. The vocabulary (often called the *lexicon*) is the following:

$V = \left\{\right.$ `a, along, and, as, at, beautiful, branches, bunch, burst, day, fox, fox's, from, gazed, grapes, hanging, he, juice, longingly, mouth, of, one, ready, ripe, seemed, spied, the, them, to, trained, tree, vine, watered, with`$\left.\right\}$

Then the document

`A Fox one day spied a beautiful bunch of ripe grapes hanging from a vine trained along the branches of a tree`

may be represented as the following sparse vector of word counts:

$$\bar V(d) = \left( 4,1,0,0,0,1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,2,1,0,1,0,0,1,0,0,1,1,1,0,0 \right)$$

or more succinctly as

`[(0, 4), (1, 1), (5, 1), (6, 1), (7, 1), (9, 1), (10, 1), (12, 1), (14, 1), (15, 1), (20, 2), (21, 1), (23, 1),`
`(26, 1), (29,1), (30, 1), (31, 1)]`

along with a dictionary

``
{
    0: a, 1: along, 5: beautiful, 6: branches, 7: bunch, 9: day, 10: fox, 12: from, 14: grapes, 15: hanging, 19: mouth, 20: of, 21: one, 23: ripe, 24: seemed, 25: spied, 26: the, 29:trained, 30: tree, 31: vine, 
}
``

Then, a set of documents becomes, in the usual `sklearn` style, a sparse matrix with rows being sparse arrays representing documents and columns representing the features/words in the vocabulary.

Notice that this representation loses the relative ordering of the terms in the document. That is "cat ate rat" and "rat ate cat" are the same. Thus, this representation is also known as the Bag-Of-Words representation.

Here is another example, from the book quoted above, although the matrix is transposed here so that documents are columns:

![novel terms](terms.png)

Such a matrix is also catted a Term-Document Matrix. Here, the terms being indexed could be stemmed before indexing; for instance, `jealous` and `jealousy` after stemming are the same feature. One could also make use of other "Natural Language Processing" transformations in constructing the vocabulary. We could use Lemmatization, which reduces words to lemmas: work, working, worked would all reduce to work. We could remove "stopwords" from our vocabulary, such as common words like "the". We could look for particular parts of speech, such as adjectives. This is often done in Sentiment Analysis. And so on. It all depends on our application.

From the book:
>The standard way of quantifying the similarity between two documents $d_1$ and $d_2$  is to compute the cosine similarity of their vector representations $\bar V(d_1)$ and $\bar V(d_2)$:

$$S_{12} = \frac{\bar V(d_1) \cdot \bar V(d_2)}{|\bar V(d_1)| \times |\bar V(d_2)|}$$

![Vector Space Model](vsm.png)


>There is a far more compelling reason to represent documents as vectors: we can also view a query as a vector. Consider the query q = jealous gossip. This query turns into the unit vector $\bar V(q)$ = (0, 0.707, 0.707) on the three coordinates below. 

![novel terms](terms2.png)

>The key idea now: to assign to each document d a score equal to the dot product:

$$\bar V(q) \cdot \bar V(d)$$

Then we can use this simple Vector Model as a Search engine.

### In Code

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text = ['Hop on pop', 'Hop off pop', 'Hop Hop hop']
print("Original text is\n{}".format('\n'.join(text)))

vectorizer = CountVectorizer(min_df=0)

# call `fit` to build the vocabulary
vectorizer.fit(text)

# call `transform` to convert text to a bag of words
x = vectorizer.transform(text)

# CountVectorizer uses a sparse array to save memory, but it's easier in this assignment to 
# convert back to a "normal" numpy array
x = x.toarray()

print("")
print("Transformed text vector is \n{}".format(x))

# `get_feature_names` tracks which word is associated with each column of the transformed x
print("")
print("Words for each feature:")
print(vectorizer.get_feature_names())

# Notice that the bag of words treatment doesn't preserve information about the *order* of words, 
# just their frequency

In [ ]:
def make_xy(critics, vectorizer=None):
    #Your code here    
    if vectorizer is None:
        vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(critics.quote)
    X = X.tocsc()  # some versions of sklearn return COO format
    y = (critics.fresh == 'fresh').values.astype(np.int)
    return X, y
X, y = make_xy(critics)

## Naive Bayes

From Bayes' Theorem, we have that

$$P(c \vert f) = \frac{P(c \cap f)}{P(f)}$$

where $c$ represents a *class* or category, and $f$ represents a feature vector, such as $\bar V(d)$ as above. **We are computing the probability that a document (or whatever we are classifying) belongs to category *c* given the features in the document.** $P(f)$ is really just a normalization constant, so the literature usually writes Bayes' Theorem in context of Naive Bayes as

$$P(c \vert f) \propto P(f \vert c) P(c) $$

$P(c)$ is called the *prior* and is simply the probability of seeing class $c$. But what is $P(f \vert c)$? This is the probability that we see feature set $f$ given that this document is actually in class $c$. This is called the *likelihood* and comes from the data. One of the major assumptions of the Naive Bayes model is that the features are *conditionally independent* given the class. While the presence of a particular discriminative word may uniquely identify the document as being part of class $c$ and thus violate general feature independence, conditional independence means that the presence of that term is independent of all the other words that appear *within that class*. This is a very important distinction. Recall that if two events are independent, then:

$$P(A \cap B) = P(A) \cdot P(B)$$

Thus, conditional independence implies

$$P(f \vert c)  = \prod_i P(f_i | c) $$

where $f_i$ is an individual feature (a word in this example).

To make a classification, we then choose the class $c$ such that $P(c \vert f)$ is maximal.

There is a small caveat when computing these probabilities. For [floating point underflow](http://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html) we change the product into a sum by going into log space. This is called the LogSumExp trick. So:

$$\log P(f \vert c)  = \sum_i \log P(f_i \vert c) $$

There is another caveat. What if we see a term that didn't exist in the training data? This means that $P(f_i \vert c) = 0$ for that term, and thus $P(f \vert c)  = \prod_i P(f_i | c) = 0$, which doesn't help us at all. Instead of using zeros, we add a small negligible value called $\alpha$ to each count. This is called Laplace Smoothing.

$$P(f_i \vert c) = \frac{N_{ic}+\alpha}{N_c + \alpha N_i}$$

where $N_{ic}$ is the number of times feature $i$ was seen in class $c$, $N_c$ is the number of times class $c$ was seen and $N_i$ is the number of times feature $i$ was seen globally. $\alpha$ is sometimes called a regularization parameter.

### Multinomial Naive Bayes and Other Likelihood Functions

Since we are modeling word counts, we are using variation of Naive Bayes called Multinomial Naive Bayes. This is because the likelihood function actually takes the form of the multinomial distribution.

$$P(f \vert c) = \frac{\left( \sum_i f_i \right)!}{\prod_i f_i!} \prod_{f_i} P(f_i \vert c)^{f_i} \propto \prod_{i} P(f_i \vert c)$$

where the nasty term out front is absorbed as a normalization constant such that probabilities sum to 1.

There are many other variations of Naive Bayes, all which depend on what type of value $f_i$ takes. If $f_i$ is continuous, we may be able to use *Gaussian Naive Bayes*. First compute the mean and variance for each class $c$. Then the likelihood, $P(f \vert c)$ is given as follows

$$P(f_i = v \vert c) = \frac{1}{\sqrt{2\pi \sigma^2_c}} e^{- \frac{\left( v - \mu_c \right)^2}{2 \sigma^2_c}}$$

<div class="span5 alert alert-info">
<h3>Exercise Set II</h3>

<p><b>Exercise:</b> Implement a simple Naive Bayes classifier:</p>

<ol>
<li> split the data set into a training and test set
<li> Use `scikit-learn`'s `MultinomialNB()` classifier with default parameters.
<li> train the classifier over the training set and test on the test set
<li> print the accuracy scores for both the training and the test sets
</ol>

What do you notice? Is this a good classifier? If not, why not?
</div>

In [ ]:
#!pip install -user --upgrade scikit-learn == 0.18.0

In [ ]:
#your turn

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
clf = MultinomialNB().fit(X_train, y_train)
print("Test Accuracy:", clf.score(X_test, y_test))
print("Train Accuracy:", clf.score(X_train,y_train))

### Picking Hyperparameters for Naive Bayes and Text Maintenance

We need to know what value to use for $\alpha$, and we also need to know which words to include in the vocabulary. As mentioned earlier, some words are obvious stopwords. Other words appear so infrequently that they serve as noise, and other words in addition to stopwords appear so frequently that they may also serve as noise.

First, let's find an appropriate value for `min_df` for the `CountVectorizer`. `min_df` can be either an integer or a float/decimal. If it is an integer, `min_df` represents the minimum number of documents a word must appear in for it to be included in the vocabulary. If it is a float, it represents the minimum *percentage* of documents a word must appear in to be included in the vocabulary. From the documentation:

>min_df: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

<div class="span5 alert alert-info">
<h3>Exercise Set III</h3>

<p><b>Exercise:</b> Construct the cumulative distribution of document frequencies (df). The $x$-axis is a document count $x_i$ and the $y$-axis is the percentage of words that appear less than $x_i$ times. For example, at $x=5$, plot a point representing the percentage or number of words that appear in 5 or fewer documents.</p>

<p><b>Exercise:</b> Look for the point at which the curve begins climbing steeply. This may be a good value for `min_df`. If we were interested in also picking `max_df`, we would likely pick the value where the curve starts to plateau. What value did you choose?</p>
</div>

In [ ]:
# Your turn.

# Isolate the reviews in the dataframe
df_words = critics["quote"]

# Put the reviews into a list 
documents = [document for document in df_words]
texts = []
for document in documents:
    texts.append(document)

In [ ]:

# Create a dictionary and tally the occurrence of each words
occurrences = {}
for text in texts:
    found_words = set([])
    for word in text.split():
        if word in found_words:
            continue
        occurrences[word] = 1 + occurrences.get(word,0)
        found_words.add(word)

sorted(occurrences, key=occurrences.__getitem__)

In [ ]:
import operator
sorted_occurrences = sorted(occurrences.items(), key=operator.itemgetter(1))
sorted_occurrences

In [ ]:
# Twenty least common terms

for i in range(20):
    print(sorted_occurrences[i])

In [ ]:
# Plot the Construct the cumulative distribution of document frequencies (df). 
# The  x-axis is a document count  xi  and the  y-axis is the percentage 
# of words that appear less than  xi  times. For example, at  x=5 , plot 
# a point representing the percentage or number of words that appear in 5 or fewer documents.

import matplotlib.pyplot as plt

def Cdf(t, x):
    count = 0.0
    for word, value in t.items():
        if value <= x:
            count += 1.0

    prob = count / len(occurrences)
    return prob

x_values = []
y_values = []

for word, value in occurrences.items():
    x_values.append(value)
    y_values.append(Cdf(occurrences, value))

plt.title("Word Frequencies in Reviews", fontsize = 20)
plt.ylabel("Probability of word occuring x times", fontsize = 15)
plt.xlabel("Number of documents a word occurs in", fontsize = 15)    
plt.scatter(x_values, y_values)
plt.xlim((0,200))
plt.show()

I limited X to 200, as the full chart (shown below) is so skewed by highly occuring words that no detail in the curve can be seen.

In [ ]:
plt.title("Word Frequencies in Reviews", fontsize = 20)
plt.ylabel("Percentage of words occuring x times", fontsize = 15)
plt.xlabel("Number of documents a word occurs in", fontsize = 15)    
plt.scatter(x_values, y_values)
plt.show()

The original question was: "Look for the point at which the curve begins climbing steeply. This may be a good value for min_df. If we were interested in also picking max_df, we would likely pick the value where the curve starts to plateau. What value did you choose?"
I chose min_df = 5, and max_df = 50

The parameter $\alpha$ is chosen to be a small value that simply avoids having zeros in the probability computations. This value can sometimes be chosen arbitrarily with domain expertise, but we will use K-fold cross validation. In K-fold cross-validation, we divide the data into $K$ non-overlapping parts. We train on $K-1$ of the folds and test on the remaining fold. We then iterate, so that each fold serves as the test fold exactly once. The function `cv_score` performs the K-fold cross-validation algorithm for us, but we need to pass a function that measures the performance of the algorithm on each fold. 

In [ ]:
from sklearn.model_selection import KFold
def cv_score(clf, X, y, scorefunc):
    result = 0.
    nfold = 5
    for train, test in KFold(nfold).split(X): # split data into train/test groups, 5 times
        clf.fit(X[train], y[train]) # fit the classifier, passed is as clf.
        result += scorefunc(clf, X[test], y[test]) # evaluate score function on held-out data
    return result / nfold # average

We use the log-likelihood as the score here in `scorefunc`. The higher the log-likelihood, the better. Indeed, what we do in `cv_score` above is to implement the cross-validation part of `GridSearchCV`.

The custom scoring function `scorefunc` allows us to use different metrics depending on the decision risk we care about (precision, accuracy, profit etc.) directly on the validation set. You will often find people using `roc_auc`, precision, recall, or `F1-score` as the scoring function.

In [ ]:
def log_likelihood(clf, x, y):
    prob = clf.predict_log_proba(x)
    rotten = y == 0
    fresh = ~rotten
    return prob[rotten, 0].sum() + prob[fresh, 1].sum()

We'll cross-validate over the regularization parameter $\alpha$.

Let's set up the train and test masks first, and then we can run the cross-validation procedure.

In [ ]:
from sklearn.model_selection import train_test_split
_, itest = train_test_split(range(critics.shape[0]), train_size=0.7)
mask = np.zeros(critics.shape[0], dtype=np.bool)
mask[itest] = True

<div class="span5 alert alert-info">
<h3>Exercise Set IV</h3>

<p><b>Exercise:</b> What does using the function `log_likelihood` as the score mean? What are we trying to optimize for?</p>

<p><b>Exercise:</b> Without writing any code, what do you think would happen if you choose a value of $\alpha$ that is too high?</p>

<p><b>Exercise:</b> Using the skeleton code below, find the best values of the parameter `alpha`, and use the value of `min_df` you chose in the previous exercise set. Use the `cv_score` function above with the `log_likelihood` function for scoring.</p>
</div>

In [ ]:
from sklearn.naive_bayes import MultinomialNB

#the grid of parameters to search over
alphas = [.1, 1, 5, 10, 50]
best_min_df = 5 # YOUR TURN: put your value of min_df here. none value to 5

#Find the best value for alpha and min_df, and the best classifier
best_alpha = None
maxscore=-np.inf
for alpha in alphas:        
    vectorizer = CountVectorizer(min_df = best_min_df)       
    Xthis, ythis = make_xy(critics, vectorizer)
    Xtrainthis = Xthis[mask]
    ytrainthis = ythis[mask]
    # your turn
    clf = MultinomialNB(alpha=alpha)
    result = cv_score(clf, Xtrainthis, ytrainthis, log_likelihood)
    if result > maxscore:
        maxscore = result
        best_alpha = alpha


In [ ]:
print("alpha: {}".format(best_alpha))

<div class="span5 alert alert-info">
<h3>Exercise Set V: Working with the Best Parameters</h3>

<p><b>Exercise:</b> Using the best value of  `alpha` you just found, calculate the accuracy on the training and test sets. Is this classifier better? Why (not)?</p>

</div>

In [ ]:
vectorizer = CountVectorizer(min_df=best_min_df)
X, y = make_xy(critics, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]

clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)

#your turn. Print the accuracy on the test and training dataset
training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest, clf.predict(xtest)))

Observation:

This classifier is quite an improvement over the previous one. That had training/test scores of 0.63 and 0.62. This one is scoring 0.75 and 0.69. Optimising the parameters as we have done has clearly made a big difference to performance. In addition, the classifier is not showing signs of over-fitting (i.e. a high training score, and a much lower test score) which is encouraging

## Interpretation

### What are the strongly predictive features?

We use a neat trick to identify strongly predictive features (i.e. words). 

* first, create a data set such that each row has exactly one feature. This is represented by the identity matrix.
* use the trained classifier to make predictions on this matrix
* sort the rows by predicted probabilities, and pick the top and bottom $K$ rows

In [ ]:
words = np.array(vectorizer.get_feature_names())

x = np.eye(xtest.shape[1])
probs = clf.predict_log_proba(x)[:, 0]
ind = np.argsort(probs)

good_words = words[ind[:10]]
bad_words = words[ind[-10:]]

good_prob = probs[ind[:10]]
bad_prob = probs[ind[-10:]]

print("Good words\t     P(fresh | word)")
for w, p in zip(good_words, good_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))
    
print("Bad words\t     P(fresh | word)")
for w, p in zip(bad_words, bad_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

<div class="span5 alert alert-info">
<h3>Exercise Set VI</h3>

<p><b>Exercise:</b> Why does this method work? What does the probability for each row in the identity matrix represent</p>

</div>

The above exercise is an example of *feature selection*. There are many other feature selection methods. A list of feature selection methods available in `sklearn` is [here](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_selection). The most common feature selection technique for text mining is the chi-squared $\left( \chi^2 \right)$ [method](http://nlp.stanford.edu/IR-book/html/htmledition/feature-selectionchi2-feature-selection-1.html).

### Prediction Errors

We can see mis-predictions as well.

In [ ]:
x, y = make_xy(critics, vectorizer)

prob = clf.predict_proba(x)[:, 0]
predict = clf.predict(x)

bad_rotten = np.argsort(prob[y == 0])[:5]
bad_fresh = np.argsort(prob[y == 1])[-5:]

print("Mis-predicted Rotten quotes")
print('---------------------------')
for row in bad_rotten:
    print(critics[y == 0].quote.iloc[row])
    print("")

print("Mis-predicted Fresh quotes")
print('--------------------------')
for row in bad_fresh:
    print(critics[y == 1].quote.iloc[row])
    print("")

<div class="span5 alert alert-info">
<h3>Exercise Set VII: Predicting the Freshness for a New Review</h3>
<br/>
<div>
<b>Exercise:</b>
<ul>
<li> Using your best trained classifier, predict the freshness of the following sentence: *'This movie is not remarkable, touching, or superb in any way'*
<li> Is the result what you'd expect? Why (not)?
</ul>
</div>
</div>

In [ ]:
#your turn

clf.predict_proba(vectorizer.transform(['This movie is not remarkable, touching, or superb in any way']))

Observation:
    
This result IS what I'd expect. Because the classifier looks at all terms as independent of each other, and therefore has no way of understanding grammatical context, it is not able to understand the function of the word "not" in this sentence. It is, therefore, easily fooled. It has picked up the "good" words remarkable, touching and superb, and judged the review to be 88.7% fresh, when in fact it is completely rotten.

### Aside: TF-IDF Weighting for Term Importance

TF-IDF stands for 

`Term-Frequency X Inverse Document Frequency`.

In the standard `CountVectorizer` model above, we used just the term frequency in a document of words in our vocabulary. In TF-IDF, we weight this term frequency by the inverse of its popularity in all documents. For example, if the word "movie" showed up in all the documents, it would not have much predictive value. It could actually be considered a stopword. By weighing its counts by 1 divided by its overall frequency, we downweight it. We can then use this TF-IDF weighted features as inputs to any classifier. **TF-IDF is essentially a measure of term importance, and of how discriminative a word is in a corpus.** There are a variety of nuances involved in computing TF-IDF, mainly involving where to add the smoothing term to avoid division by 0, or log of 0 errors. The formula for TF-IDF in `scikit-learn` differs from that of most textbooks: 

$$\mbox{TF-IDF}(t, d) = \mbox{TF}(t, d)\times \mbox{IDF}(t) = n_{td} \log{\left( \frac{\vert D \vert}{\vert d : t \in d \vert} + 1 \right)}$$

where $n_{td}$ is the number of times term $t$ occurs in document $d$, $\vert D \vert$ is the number of documents, and $\vert d : t \in d \vert$ is the number of documents that contain $t$

In [ ]:
# http://scikit-learn.org/dev/modules/feature_extraction.html#text-feature-extraction
# http://scikit-learn.org/dev/modules/classes.html#text-feature-extraction-ref
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfvectorizer = TfidfVectorizer(min_df=1, stop_words='english')
Xtfidf=tfidfvectorizer.fit_transform(critics.quote)

<div class="span5 alert alert-info">
<h3>Exercise Set VIII: Enrichment <b>(Optional)</b></h3>

<p>
There are several additional things we could try. Try some of these as exercises:
<ol>
<li> Build a Naive Bayes model where the features are n-grams instead of words. N-grams are phrases containing n words next to each other: a bigram contains 2 words, a trigram contains 3 words, and 6-gram contains 6 words. This is useful because "not good" and "so good" mean very different things. On the other hand, as n increases, the model does not scale well since the feature set becomes more sparse.
<li> Try a model besides Naive Bayes, one that would allow for interactions between words -- for example, a Random Forest classifier.
<li> Try adding supplemental features -- information about genre, director, cast, etc.
<li> Use word2vec or [Latent Dirichlet Allocation](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) to group words into topics and use those topics for prediction.
<li> Use TF-IDF weighting instead of word counts.
</ol>
</p>

<b>Exercise:</b> Try at least one of these ideas to improve the model (or any other ideas of your own). Implement here and report on the result. 
</div>

In [ ]:
# Your turn

def find_ngrams(input_list, n):
    return zip(*[input_list[i:] for i in range(n)])

In [ ]:
# Create a dictionary and tally the occurrence of each words
ngram_occurrences = {}
for text in texts:
    ngrams = find_ngrams(text.split(), 2)
    found_ngrams = set([])
    for ngram in ngrams:
        if ngram in found_ngrams:
            continue
        ngram_occurrences[ngram] = 1 + ngram_occurrences.get(ngram,0)
        found_ngrams.add(ngram)

In [ ]:
ngram_occurrences

In [ ]:
import operator
sorted_ngram_occurrences = sorted(ngram_occurrences.items(), key=operator.itemgetter(1))

# Twenty most common ngrams
for i in range(20):
    if i > 0: 
        print(sorted_ngram_occurrences[-i])

print("\n")
        
# Twenty least common terms
for i in range(20):
    print(sorted_ngram_occurrences[i])

In [ ]:
# Plot the Construct the cumulative distribution of document frequencies (df). 
# The  x-axis is a document count  xi  and the  y-axis is the percentage 
# of words that appear less than  xi  times. For example, at  x=5 , plot 
# a point representing the percentage or number of words that appear in 5 or fewer documents.

import matplotlib.pyplot as plt

def ngramCdf(t, x):
    count = 0.0
    for ngram, value in t.items():
        if value <= x:
            count += 1.0

    prob = count / len(ngram_occurrences)
    return prob

x_values = []
y_values = []

for ngram, value in ngram_occurrences.items():
    x_values.append(value)
    y_values.append(ngramCdf(ngram_occurrences, value))

plt.title("Word Frequencies in Reviews", fontsize = 20)
plt.ylabel("Probability of word occuring x times", fontsize = 15)
plt.xlabel("Number of documents a word occurs in", fontsize = 15)    
plt.scatter(x_values, y_values)
#plt.xlim((0,200))
plt.show()

In [ ]:
plt.title("Word Frequencies in Reviews", fontsize = 20)
plt.ylabel("Probability of word occuring x times", fontsize = 15)
plt.xlabel("Number of documents a word occurs in", fontsize = 15)    
plt.scatter(x_values, y_values)
plt.xlim((0,200))
plt.show()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Create the grid of parameters to search over
alphas = [.1, 1, 5, 10, 50]
best_min_df = 5 # YOUR TURN: put your value of min_df here.

# Find the best value for alpha and min_df, and the best classifier
best_alpha = None
maxscore=-np.inf
for alpha in alphas:        
    vectorizer = CountVectorizer(ngram_range=(1, 5), max_df=50, min_df=5)       
    Xthis, ythis = make_xy(critics, vectorizer)
    Xtrainthis = Xthis[mask]
    ytrainthis = ythis[mask]
    # your turn
    clf = MultinomialNB(alpha=alpha)
    result = cv_score(clf, Xtrainthis, ytrainthis, log_likelihood)
    if result > maxscore:
        maxscore = result
        best_alpha = alpha
        
print("alpha: {}".format(best_alpha))

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 5), max_df=50, min_df=5) 
X, y = make_xy(critics, vectorizer)
xtrain=X[mask]
ytrain=y[mask]
xtest=X[~mask]
ytest=y[~mask]


clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)

#your turn. Print the accuracy on the test and training dataset
training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: {:2f}".format(training_accuracy))
print("Accuracy on test data:     {:2f}".format(test_accuracy))

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: %0.2f" % (training_accuracy))
print("Accuracy on test data:     %0.2f" % (test_accuracy))

predicted = clf.predict(xtest)
print("\nMean prediction accuracy is: ", np.mean(predicted == ytest))
print("\n\nClassification report:\n", metrics.classification_report(ytest, predicted))
print("\n\nConfusion matrix:\n", metrics.confusion_matrix(ytest, predicted))

In [ ]:
words = np.array(vectorizer.get_feature_names())

x = np.eye(xtest.shape[1])
probs = clf.predict_log_proba(x)[:, 0]
ind = np.argsort(probs)

good_words = words[ind[:10]]
bad_words = words[ind[-10:]]

good_prob = probs[ind[:10]]
bad_prob = probs[ind[-10:]]

print("Good words\t     P(fresh | word)")
for w, p in zip(good_words, good_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))
    
print("Bad words\t     P(fresh | word)")
for w, p in zip(bad_words, bad_prob):
    print("{:>20}".format(w), "{:.2f}".format(1 - np.exp(p)))

In [ ]:
clf.predict_proba(vectorizer.transform(['This movie is not remarkable, touching, or superb in any way']))

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

vectorizer = CountVectorizer()       
Xthis, ythis = make_xy(critics, vectorizer)
Xtrain=Xthis[mask]
ytrain=ythis[mask]
Xtest=Xthis[~mask]
ytest=ythis[~mask]

rf = RandomForestClassifier(max_depth=10,max_features=100)
clf = rf.fit(X_train,y_train)

In [ ]:
training_accuracy = clf.score(Xtrain, ytrain)
test_accuracy = clf.score(Xtest, ytest)

print("Accuracy on training data: %0.2f" % (training_accuracy))
print("Accuracy on test data:     %0.2f" % (test_accuracy))

predicted = clf.predict(Xtest)
print("\nMean prediction accuracy is: ", np.mean(predicted == ytest))
print("\n\nClassification report:\n", metrics.classification_report(ytest, predicted))
print("\n\nConfusion matrix:\n", metrics.confusion_matrix(ytest, predicted))

4. Use word2vec or Latent Dirichlet Allocation to group words into topics and use those topics for prediction.¶

In [ ]:
#!pip3 install NLTK

import nltk
nltk.download('stopwords')

In [ ]:
# Load body content from folder of classified jsons and create bag-of-words corpus"""

import time
import gensim
import os

import pandas as pd
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer

#import clean_text
import textcleaner as tc 


wnl = WordNetLemmatizer()
start_time = time.time()

# create a list 'prepped_texts' to store the review content for creating the corpus
prepped_texts = []
filenumber = 1  # File counter for use when iterating files and registering file ID

# pull out the review content and add it to the documents list
for text in texts:
    #prepped_text = clean_text.clean_text(text)
    prepped_text = tc.textcleaner.
    
    # Add the article to the list of all articles
    prepped_texts.append(prepped_text)
        #filenumber += 1

# print('\nCleaned and lemmatized corpus texts:\n\n', prepped_texts)

# Create the dictionary
dictionary = corpora.Dictionary(prepped_texts)

# Filter out low frequency words and compact AFTER dictionary is created (preferred method)
dictionary.filter_extremes(no_below=5, no_above=0.5)
dictionary.compactify()

# Save the filtered dictionary, for future reference
dictionary.save(os.path.join("./lda/", "_bow_corpus.dict"))
print("Dictionary:", dictionary, "\n")

# Create and save the corpus
corpus = [dictionary.doc2bow(prepped_text) for prepped_text in prepped_texts]

# Save to disk, for later use
corpora.MmCorpus.serialize((os.path.join("./lda/", "_bow_corpus.mm")), corpus)

print("Finished bag-of-words corpus and dictionary creation, took {}".format(time.time() - start_time))

In [ ]:
# Define the dictionary and corpus variables with the dictionary 
# and corpus we've just created, and create a model with 2 topics

import gensim
from gensim import corpora
from gensim.models import LdaMulticore 

dictionary = corpora.Dictionary.load('./lda/_bow_corpus.dict')
corpus = corpora.MmCorpus('./lda/_bow_corpus.mm')

model = LdaMulticore(corpus, id2word=dictionary, num_topics=2)
model.save('./lda/all_topics.lda')
model.print_topics(-1)

In [ ]:
# Define a function to handle a query procedure
# matching a given text to the LDA topics

def comparison(query):
    query = query.split()
    query = dictionary.doc2bow(query)
    return query

# Run a query 

query = 'This movie is not remarkable, touching, or superb in any way'
vector = comparison(query)
model[vector]
a = list(sorted(model[vector], key=lambda x: x[1]))
print(a[0])
print(model.print_topic(a[0][0])) #least related
print(a[-1])
print(model.print_topic(a[-1][0])) #most related

In [ ]:
# Extract 2 LDA topics using 20 full passes

model = LdaMulticore(corpus, id2word=dictionary, num_topics=2, passes=20)
model.save('./lda/all_topicsx20.lda')
model.print_topics(-1)

In [ ]:
# Run a query 

query = 'This movie is not remarkable, touching, or superb in any way'
vector = comparison(query)
model[vector]
a = list(sorted(model[vector], key=lambda x: x[1]))
print(a[0])
print(model.print_topic(a[0][0])) #least related
print(a[-1])
print(model.print_topic(a[-1][0])) #most related

In [ ]:
# Run a query 

query = 'Michael Mann and a superlative cast have taken a classic heist movie rife with familiar genre elements and turned it into a sleek, accomplished piece of work, meticulously controlled and completely involving.'
vector = comparison(query)
model[vector]
a = list(sorted(model[vector], key=lambda x: x[1]))
print(a[0])
print(model.print_topic(a[0][0])) #least related
print(a[-1])
print(model.print_topic(a[-1][0])) #most related

In [ ]:
print(model[vector])

In [ ]:
a = sorted(model[vector])
print(a)

In [ ]:
v1,v2 = a[0]
print(v2)

In [ ]:
# Create vectors for the lda topic distributions for each review

lda_vectors = []

for text in texts:
    vector = comparison(text)
    a = model[vector]
    b = []
    v1,v2 = a[0]
    v3,v4 = a[1]
    b = [v2, v4]
    lda_vectors.append(b)

print(lda_vectors)